In [ ]:
# This block is personal notes from various stages of the project
# This primarily served as a TODO list where I could also sketch
# basic ideas of how to solve upcoming problems



# TODO:
# Prove whether or not current method actually works,
# collision avoidance, termination, etc.
# AND / OR
# Write movement step code
# Use random boards as 'proof' of concept

# Get perform move working
# Representation of paths in code
# -> transition towards pre-calculating paths

# For 3d - assemble first layer into sheet, then do second on top

# At some point, must get conversion between startid and local botid;
# possibly from manual setup in desired locations
# (since we can't currently get position from bot)

# GO THROUGH ENTIRE PROJECT AND DOUBLE CHECK:
# ARE XPOS, YPOS, ROW, COL APPROPRIATELY DONE FOR BOARDS

# When revising assembly algorithm, can check:
# if position ahead that's sticking is end position, fail that branch
# if not, wait one unit of time, then check again
# Note this method can be optimized by waiting bot being one with smaller distance

# Transfer boards to use 0 or id
# Have start board read ids and place into dict differently
# end board read should be the same though, as 1 and 10 not related
# Then, can create start board from bot pos OR bot pos from start board
# rather than just bot pos from start board (as ids are pre-assigned)
# Helps with id initialization, but not position
# if position is solved, then can extrapolate from there
# Uglier solution, but could automate process by picture to board,
# but gonna be more effort than it's worth

# Next time: Test functionality with different board sizes and bot counts;
# checking indexing in all functions
# Also, work on adding alternative pathfinding measures for cases where we fail



# Write latching algorithm - need code for bots to stick together once assembled
# With our hardware (electromagnets), need to turn on only one side of each connection
# so that side attracts the metal in the other turned off magnet
# This should work better with electropermanent magnets, which have more metal
# Ideally, use magnets which can be positive or negative (or off), allowing for
# much more attractive force

# Latching Algorithm: Flag final move as final position
# At end of step, check all bots that just reached final position
#    (store what has already reached and check against that a la graph algo?)
# Of bots that just reached final position,
#   check if touching an already reached final position bot;
#   if so, current bot turns that face on
# If touching bot that also just reached final position,
# somehow determine one to have face on and other to not
# (move first one that latching algo found to already reached
#   final position set after latching to all already reached)

<h1>Setup</h1>

In [ ]:
import numpy as np
import itertools
import random
import time
from IPython.display import clear_output

In [ ]:
# Setup
xsize = 5
ysize = 5
size = (xsize, ysize)

board = [[0,1,0,0,0],[0]*5,[0,0,0,1,0],[0,0,1,0,0],[1,0,0,1,0]]
board = np.asarray(board)
print(board)

[[0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [1 0 0 1 0]]


In [ ]:
# Randomly generate a board

def gen_board(xsize, ysize, bots):
  # Random ints between 0 and x*y-1, format to array
  startlocs = random.sample(range(xsize * ysize), bots)
  board = []
  for row in range(ysize):
    board.append([0] * xsize)
  row, col = 0, 0
  for val in list(startlocs):
    # Would be faster to do division with remainder to get row, val
    while val >= xsize:
      row += 1
      val -= xsize
    col = val
    board[row][col] = 1
    row, col = 0, 0
  board = np.asarray(board)
  return board

In [ ]:
print(gen_board(5, 5, 5))

[[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 1 0 1]
 [0 0 0 0 0]
 [0 0 0 0 1]]


In [ ]:
# Create an ending board manually; a cross in the middle
end_board = [ [0]*5, [0,0,1,0,0], [0,1,1,1,0], [0,0,1,0,0], [0]*5]
end_board = np.asarray(end_board)
print(end_board)

[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 1 1 1 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]


In [ ]:
board = gen_board(5, 5, 5)
end_board = gen_board(5, 5, 5)
print(board)
print(end_board)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]
 [0 1 0 0 1]
 [0 0 1 0 1]]
[[0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [0 0 1 1 0]]


In [ ]:
# Loop through board, cols then rows, finding 1's, id of order found in to (x, y)
# If isend, id's scale in 10s instead of 1s for convenience?

# Scaling id by 10 this way only works when less than 10 bots
# Soln: multiply by power of 10 to prevent overlap (sum to get bot count?)

# NOTE: IF WE CHANGE startids FROM BEING 1,2,3,...
# THEN MANY LOOPS LATER IN THE CODE WILL BREAK
# (often loop through tuple 0,1,2... and through dict 0+1,1+1,2+1,...)
def convert_board(board, isend):
  ident = 1
  if isend:
    ident = 10
  botpos = {}
  for row in range(len(board)):
    for col in range(len(board[0])):
      if board[row, col] == 1:
        botpos[ident] = (row, col)
        ident += 1
        if isend:
          ident += 9
  return botpos

In [ ]:
# Using boards that errored, randomly generated
"""
print(savestartboard)
print(saveendboard)
startlist = convert_board(savestartboard, False)
endlist = convert_board(saveendboard, True)
print(startlist)
print(endlist)
"""

'\nprint(savestartboard)\nprint(saveendboard)\nstartlist = convert_board(savestartboard, False)\nendlist = convert_board(saveendboard, True)\nprint(startlist)\nprint(endlist)\n'

In [ ]:
startlist = convert_board(board, False)
print(startlist)
endlist = convert_board(end_board, True)
print(endlist)

{1: (0, 1), 2: (2, 3), 3: (3, 2), 4: (4, 0), 5: (4, 3)}
{10: (1, 2), 20: (2, 1), 30: (2, 2), 40: (2, 3), 50: (3, 2)}


In [ ]:
# For each bot position, find distance to each end position
# Returns dict of startid to dict of endid to distance from startpos to endpos
def find_dists(startlist, endlist):
  dists = {}
  for sid in startlist:
    newlist = {}
    srow, scol = startlist[sid]
    for eid in endlist:
      erow, ecol = endlist[eid]
      # Distance is difference in x coord + difference in y coord
      newlist[eid] = abs(erow - srow) + abs(ecol - scol)
    dists[sid] = newlist
  return dists

In [ ]:
distlist = find_dists(startlist, endlist)
print(distlist)

{1: {10: 2, 20: 2, 30: 3, 40: 4, 50: 4}, 2: {10: 2, 20: 2, 30: 1, 40: 0, 50: 2}, 3: {10: 2, 20: 2, 30: 1, 40: 2, 50: 0}, 4: {10: 5, 20: 3, 30: 4, 40: 5, 50: 3}, 5: {10: 4, 20: 4, 30: 3, 40: 2, 50: 2}}


<h1>Pathfinding</h1>

In [ ]:
# Figure out more efficient pathfinding
# Ideas:
# Zero distance is probably better off moving out of the way of another
# If we can calculate minimum sum across permutations of ids, then we might be able to find that
# ^^ Might solve the issue above, but want to prove
# Also, need to write the code for it

In [ ]:
# Given 2d array, where each row is a starting position and each column is
  # distance to a given end position,
# want to find unique columns for each row such that the sum
  # of the distances is minimized
# Could brute force by checking each permutation, but that runs in n factorial
  # (size of S_n) which is absurd
# Is it possible to make divide and conquer work?
# Feels like no, since we could easily get overlap:
# [ [1, 2, 3, 4], [2, 1, 3, 4], [0, 5, 1, 2], [1, 4, 3, 2] ]
  # has overlapping indices regardless of splitting rows or cols
  # (Splitting cols gives indices 0, 1 : 0, 3 OR 2, 0;
    # splitting rows gives indices 2, 1 : 2, 3)

In [ ]:
# One possible method (still computationally expensive)
# Transform dict of dict into array of dict
distarr = np.array([distlist[key + 1] for key in range(len(distlist))]).T
# Transform array of dict into array of array
distarr = np.asarray([list(row.values()) for row in distarr])
print(distarr)
# Now arr is 2d array, where (i, j) = (i+1, (j+1)*10)

[[2 2 3 4 4]
 [2 2 1 0 2]
 [2 2 1 2 0]
 [5 3 4 5 3]
 [4 4 3 2 2]]


In [ ]:
# Given array, returns permutation of column indices
# to get minimum sum of values
# Returns tuple where ith value is the column for the ith row
# TODO: Write case where no permutations
def minsum(arr):
  # Set large minimum as default
  mini = np.max(arr) * len(arr)

  # For each possible permutation,
  for perm in itertools.permutations(range(len(arr))):
    total = 0
    # Add the values at (i, perm[i]) to sum
    for i in range(len(arr)):
      total += arr[i][list(perm)[i]]
      if total > mini:
        break
    # If perm has smaller sum than min, store new min sum and permutation
    if total < mini:
      mini = total
      minperm = perm
  # Return the smallest permutation found
  return minperm

  # OLD:
  """
  for i in range(ysize):
      # HERE we can implement periodic sum checking
      lst.append(arr[i][list(perm)[i]])
    # If perm has smaller sum than min, store new min sum and permutation
    if sum(lst) < min:
      min = sum(lst)
      minperm = perm
  """

In [ ]:
# Note: I don't know if this method will work overall,
  # since we might still be able to create traffic jams
  # Validity of algorithm must be proven,
  # but likely best to improve first
  # Proof might help visualization/learning how to make working algo

print(minsum(distarr))

(0, 2, 4, 1, 3)


<h1> TESTING</h1>

In [ ]:
# Takes tuple of end goal positions and dict of endid : endpos
# Returns dict from startid : endpos
def convert_goals(goals, endlist):
  goaldict = {}
  ident = 1
  for pos in goals:
    goaldict[ident] = endlist[(pos + 1) * 10]
    ident += 1
  return goaldict

In [ ]:
# Note that at this point, endid is no longer used
# We have startid based on startpos and endpos as identifiers
goals = convert_goals(minsum(distarr), endlist)
print(goals)

{1: (1, 2), 2: (2, 2), 3: (3, 2), 4: (2, 1), 5: (2, 3)}


In [ ]:
# print(savestartboard)
# print(saveendboard)

In [ ]:
# Get start position and end position from start and goal
"""
# Idea: create binary tree, node has position, left side being hori, right side being verti
# Create tree for each start and end pair
# Traverse each path possibility and check for collisions

COUNT = 0

class Node:
  def __init__(self, hori, verti, pos, isEnd):
    self.hori = hori
    self.verti = verti
    self.pos = pos
    self.isEnd = isEnd

  # I don't even know why this doesn't work
  def str(self):
    global COUNT
    COUNT += 1 # Hopefully this works as intended
    c = COUNT
    print("Node",str(c),self.pos)
    h = "None"
    if self.hori is not None:
      h = self.hori.str()
    print("hori ("+str(c)+"): "+h)
    v = "None"
    if self.verti is not None:
      v = self.verti.str()
    print("verti ("+str(c)+"): "+v)

# Creates tree of all possible (direct) paths from given start and end position
# None means going further in that direction is away from end
# isEnd flag indicates ending nodes
def path(start, end):
  xs, ys = start
  xe, ye = end
  # If we are at end, no need to add another node
  if start == end:
    return Node(None, None, end, True)
  # Otherwise, add node with next in vertical and horizontal path
  if xs < xe:
    hori = path((xs+1, ys), end)
  elif xs > xe:
    hori = path((xs-1, ys), end)
  else:
    hori = None
  if ys < ye:
    verti = path((xs, ys+1), end)
  elif ys > ye:
    verti = path((xs, ys-1), end)
  else:
    verti = None
  return Node(hori, verti, start, False)
"""
# Rather than using clunky and slow OOP trees, just go straight from lists
# We pass in lsts as an empty list; function populates lsts with desired paths
def pathList(start, end, lst, lsts):
  srow, scol = start
  erow, ecol = end
  if start == end:
    lsts.append(lst + [end])
    return
  if srow < erow:
    pathList((srow+1, scol), end, lst + [start], lsts)
  elif srow > erow:
    pathList((srow-1, scol), end, lst + [start], lsts)
  if scol < ecol:
    pathList((srow, scol+1), end, lst + [start], lsts)
  elif scol > ecol:
    pathList((srow, scol-1), end, lst + [start], lsts)
  # To add in more complexity (kinda brute forcing better results),
  # add in pathList((srow, scol), end, lst+[start], lsts)
  # BUT add a flag so that we only a finite number of pauses;
  # otherwise we will make infinite lists
  # add parameter hasPaused; if not hasPaused: add in a pause

# This is tree-based code, deprecated
# root = path((1,1), (2,2))
# print(root.str())
# root2 = path((3,4),(0,0))

lsts = []
pathList((3,4), (0,0), [], lsts)
print(lsts)
print(len(lsts))

# So, this works for getting ONE tree out of a start and end
# Whether that tree is useful or not is yet to be seen
# And, how to transform that tree into something useful

# I think the next step is to transform the tree into lists
# Where we have positions and time intervals (indicies)
# Then, we look at every bot's position simultaneously,
# choose some paths, and check for collisions at all times
# If no collisions, good to go
# Otherwise, change the colliding paths somehow

[[(3, 4), (2, 4), (1, 4), (0, 4), (0, 3), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (1, 4), (1, 3), (0, 3), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (1, 4), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (0, 1), (0, 0)], [(3, 4), (2, 4), (1, 4), (1, 3), (1, 2), (1, 1), (1, 0), (0, 0)], [(3, 4), (2, 4), (2, 3), (1, 3), (0, 3), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (1, 3), (1, 2), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (1, 3), (1, 2), (1, 1), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (1, 3), (1, 2), (1, 1), (1, 0), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (1, 2), (0, 2), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (1, 2), (1, 1), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (1, 2), (1, 1), (1, 0), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (2, 1), (1, 1), (0, 1), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (2, 1), (1, 1), (1, 0), (0, 0)], [(3, 4), (2, 4), (2, 3), (2, 2), (2, 1), (2, 0), (1, 0), (0, 0)], [(3, 4), 

In [ ]:
# Create a dictionary id maps to list of all possible paths from start to goal
def getAllPaths(startlist, goals):
  allPaths = {}
  for id in goals:
    lsts = []
    pathList(startlist[id], goals[id], [], lsts)
    allPaths[id] = lsts.copy()
  return allPaths
# print(startlist)
# print(goals)
allPaths = getAllPaths(startlist, goals)
print(allPaths)

{1: [[(0, 1), (1, 1), (1, 2)], [(0, 1), (0, 2), (1, 2)]], 2: [[(2, 3), (2, 2)]], 3: [[(3, 2)]], 4: [[(4, 0), (3, 0), (2, 0), (2, 1)], [(4, 0), (3, 0), (3, 1), (2, 1)], [(4, 0), (4, 1), (3, 1), (2, 1)]], 5: [[(4, 3), (3, 3), (2, 3)]]}


In [ ]:
# We calculate at what time stamp each bot will be in its final position
# This will be necessary to store for the latching algorithm later
def finalPosTimes(allPaths):
  return [len(allPaths[i][0])-1 for i in allPaths.keys()]

finalTimes = finalPosTimes(allPaths)
print(finalTimes)

[2, 1, 0, 3, 2]


In [ ]:
# We'll also want to convert allPaths to a more helpful format,
# so we change all lists to the largest length and repeat the last entry to fill
# First, find largest value
def convertAllPaths(allPaths):
  # Also, make more pythonic? do list comprehension then find max?
  maxi = -1
  for paths in allPaths.values():
    newval = len(paths[0]) # will have to change if we add in waits;
    # right now assumes that all paths are same length
    # ^ might be problem currently happening; some lists are diff lengths
    # though they shouldn't be at this current moment
    # Make a fully comprehensive maxi search, see if that resolves
    if newval > maxi:
      maxi = newval
  for paths in allPaths.values():
    # paths is lst of lst of tuples
    for i in range(len(paths)):
      lastval = paths[i][len(paths[i]) - 1]
      for j in range(maxi - len(paths[i])):
        paths[i].append(lastval)
  return maxi
maxi = convertAllPaths(allPaths)
print(maxi, allPaths)

4 {1: [[(0, 1), (1, 1), (1, 2), (1, 2)], [(0, 1), (0, 2), (1, 2), (1, 2)]], 2: [[(2, 3), (2, 2), (2, 2), (2, 2)]], 3: [[(3, 2), (3, 2), (3, 2), (3, 2)]], 4: [[(4, 0), (3, 0), (2, 0), (2, 1)], [(4, 0), (3, 0), (3, 1), (2, 1)], [(4, 0), (4, 1), (3, 1), (2, 1)]], 5: [[(4, 3), (3, 3), (2, 3), (2, 3)]]}


In [ ]:
# Next, we want to check all paths to see if any combinations work without collision
# To do this, if we can generate all combinations of indices for each id,
# then we can check the paths by checking index combinations
def makeLsts(allPaths):
  pathLengths = [len(x) for x in allPaths.values()]
  # print(pathLengths)
  rangeList = [range(x) for x in pathLengths]
  # print(rangeList)
  lsts = list(itertools.product(*rangeList))
  return lsts
lsts = makeLsts(allPaths)
print(lsts)
# Now, lsts contains all possible pairs of lists to check against each other

[(0, 0, 0, 0, 0), (0, 0, 0, 1, 0), (0, 0, 0, 2, 0), (1, 0, 0, 0, 0), (1, 0, 0, 1, 0), (1, 0, 0, 2, 0)]


In [ ]:
# So, we check if any of our lists work together
# This returns the indices of the first set of working paths found
def getIndices(allPaths, lsts, maxi):
  for indices in lsts:
    # Reform allPaths into list of lists with the chosen paths
    currpaths = []
    for i in range(len(allPaths)):
      currpaths.append(allPaths[i + 1][indices[i]])
    indfail = False
    for i in range(maxi):
      if not indfail:
        timeslice = []
        for j in range(len(currpaths)):
          timeslice.append(currpaths[j][i])
        # Compare number of bots with number of unique positions occupied,
        # inequality means overlap, thus failure
        if len(timeslice) != len(set(timeslice)):
          # print("Failure on time", i)
          indfail = True
    if not indfail:
      return indices
  # If nothing was returned by this point, we failed to match
  raise Exception("No paths found")
indices = getIndices(allPaths, lsts, maxi)
print(indices)

(0, 0, 0, 0, 0)


In [ ]:
# Now that we know the paths, we can use this to generate movement instructions
def getFinalPaths(allPaths, indices):
  finalPaths = {}
  for i in range(len(allPaths)):
    finalPaths[i+1] = allPaths[i+1][indices[i]]
  return finalPaths
print(getFinalPaths(allPaths, indices))

{1: [(0, 1), (1, 1), (1, 2), (1, 2)], 2: [(2, 3), (2, 2), (2, 2), (2, 2)], 3: [(3, 2), (3, 2), (3, 2), (3, 2)], 4: [(4, 0), (3, 0), (2, 0), (2, 1)], 5: [(4, 3), (3, 3), (2, 3), (2, 3)]}


In [ ]:
# Using final paths, make animation of bots moving
import time
import matplotlib.pyplot as plt

finalPaths = getFinalPaths(allPaths, indices)

# NOTE: animation needs to be given board dimensions
def animation(finalPaths):
  pathArr = [path for path in finalPaths.values()]
  print(board)
  for step in range(1, len(pathArr[0])):
    newboard = np.zeros((5,5)).astype(int)
    for bot in range(len(pathArr)):
        row, col = pathArr[bot][step]
        newboard[row, col] = 1
    # clear_output(wait=True)
    time.sleep(1)
    print(newboard)

def plotmaker(finalPaths):
  pathArr = [path for path in finalPaths.values()]
  # Convert to loop that produces single images, then save them
  fig, ax = plt.subplots(1,len(pathArr[0]))
  fig.set_size_inches(30,10)
  ax[0].imshow(board, cmap='binary')
  for step in range(len(pathArr[0])):
    newboard = np.zeros((5,5)).astype(int)
    for bot in range(len(pathArr)):
        row, col = pathArr[bot][step]
        oldrow, oldcol = pathArr[bot][step - 1]
        newboard[row, col] = 1
        if step != 0:
          ax[step - 1].annotate('',
              xy=(col, row), xycoords='data',
              xytext=(oldcol, oldrow), textcoords='data',
              arrowprops=dict(facecolor='red', shrink=0.05),
              horizontalalignment='right', verticalalignment='top')
    ax[step].imshow(newboard, cmap='binary')
    ax[step].title.set_text('Time step ' + str(step))
    # fig.savefig('demo_hori.png', transparent=True)

animation(finalPaths)
# plotmaker(finalPaths)
# ['34111', '20110', '70128', '00060']
# L4 R2 S3 N1

[[0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [1 0 0 1 0]]
[[0 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [1 0 1 1 0]
 [0 0 0 0 0]]
[[0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 1 1 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]
[[0 0 0 0 0]
 [0 0 1 0 0]
 [0 1 1 1 0]
 [0 0 1 0 0]
 [0 0 0 0 0]]


In [ ]:
# Convert from finalPaths to BLE information

# The following loops over time, increasin index for each path
# First, for each bot, check current position v next position,
# determine what direction we're moving
# Then, in our output, in position corresponding to id, write direction as number
def getBLEInstructions(finalPaths, maxi, finalTimes):
  steplist = []
  finals = []
  for time in range(maxi):
    direction = ""
    justFinals = []
    for ident in finalPaths:
      # Getting issue where maxi > len(finalPaths[ident]),
      # but those should be the same value
      # print("maxi: ", maxi, "finalPaths[ident]: ", finalPaths[ident])
      # Need to fix these to row, col and rewrite direction as needed
      srow, scol = finalPaths[ident][time]
      if time != maxi - 1:
        erow, ecol = finalPaths[ident][time + 1]
      if time != maxi - 1 and scol > ecol: # LEFT
        direction += '4'
      elif time != maxi - 1 and scol < ecol: # RIGHT
        direction += '2'
      elif time != maxi - 1 and srow < erow: # SOUTH
        direction += '3'
      elif time != maxi - 1 and srow > scol: # NORTH
        direction += '1'
      else: # STATIONARY
        direction += '0'

# Does the original version even properly give enough instructions?

        # Need some way of adding this latching as end step of longest path
        # Could just add a duplicate end to the paths when making them?

        # Also, test latching

        if time == finalTimes[ident - 1] and ident not in finals: # If ident just performed final move
          justFinals += [ident] # Store ident as reaching final position
      for i in justFinals: # For each bot just reaching final position,
        row, col = finalPaths[i][time]
        for otherId in finals: # If touching a bot in final position,
          # turn the face touching the other bot on to latch
          otherRow, otherCol = finalPaths[otherId][time]
          if otherRow - row == 1: # Other is south
            direction = direction[:i-1] + '7' + direction[i:]
          if otherRow - row == -1: # Other is north
            direction = direction[:i-1] + '5' + direction[i:]
          if otherCol - col == 1: # Other is right
            direction = direction[:i-1] + '6' + direction[i:]
          if otherCol - col == -1: # Other is left
            direction = direction[:i-1] + '8' + direction[i:]
        # Now that i is in final position, remove from arriving list,
        # add to stationary list so other arriving bots can latch to it
        justFinals.remove(i)
        finals += [i]

    # What datatype should direction be? String? bytearray?
    # It's being read as a string, so if we send it as a string,
    # then we get exactly what we want
    # and can read char by char as desired
    steplist += [direction]
  return steplist

""" OLD
# Note, we use range(max-1) since we only need max-1 moves
for time in range(max - 1):
  direction = []
  for id in finalPaths:
    ys, xs = finalPaths[id][time]
    ye, xe = finalPaths[id][time + 1]
    if xs > xe: # LEFT
      direction += [4]
    elif xs < xe: # RIGHT
      direction += [2]
    elif ys < ye: # SOUTH
      direction += [3]
    elif ys > ye: # NORTH
      direction += [1]
    else: # STATIONARY
      direction += [0]
  print(direction)
"""
print(getBLEInstructions(finalPaths, maxi, finalTimes))

['34111', '20110', '70128', '00060']


In [ ]:
def getBLEInstructions(finalPaths, maxi):
  steplist = []
  for time in range(maxi - 1):
    direction = ""
    for ident in finalPaths:
      # Getting issue where maxi > len(finalPaths[ident]),
      # but those should be the same value
      # print("maxi: ", maxi, "finalPaths[ident]: ", finalPaths[ident])
      # Need to fix these to row, col and rewrite direction as needed
      srow, scol = finalPaths[ident][time]
      erow, ecol = finalPaths[ident][time + 1]
      if scol > ecol: # LEFT
        direction += '4'
      elif scol < ecol: # RIGHT
        direction += '2'
      elif srow < erow: # SOUTH
        direction += '3'
      elif srow > scol: # NORTH
        direction += '1'
      else: # STATIONARY
        direction += '0'

    # What datatype should direction be? String? bytearray?
    # It's being read as a string, so if we send it as a string,
    # then we get exactly what we want
    # and can read char by char as desired
    steplist += [direction]
  return steplist
print(getBLEInstructions(finalPaths, maxi))

['34111', '20110', '00120']


In [ ]:
# Sending via bluetooth:
# Make sure IDs of each bot in this notebook and on bots match
  # ^^ Need to manually place/check? Since we don't have relative position botside
# Look at the direction each bot will move in
# Publish to characteristic a message where direction is 00,01,10,11
  # and bot to move that direction is position ID in the message
# (This condenses all relevant information into most concise format)

In [ ]:
# Here's a black box for the whole process
# Takes in startboard, endboard, returns dict from startid to path
def blackbox(startboard, endboard):
  startlist = convert_board(startboard, False)
  endlist = convert_board(endboard, True)
  distlist = find_dists(startlist, endlist)
  distarr = np.array([distlist[key + 1] for key in range(len(distlist))]).T
  distarr = np.asarray([list(row.values()) for row in distarr])
  mins = minsum(distarr)
  goals = convert_goals(mins, endlist)
  allPaths = getAllPaths(startlist, goals)
  maxi = convertAllPaths(allPaths)
  lsts = makeLsts(allPaths)
  indices = getIndices(allPaths, lsts, maxi)
  finalPaths = getFinalPaths(allPaths, indices)
  steplist = getBLEInstructions(finalPaths, maxi)
  # Change using indices -> paths
  return finalPaths
  # return steplist

In [ ]:
def rando_test(rows, cols, bots, iters):
  count = 0
  for i in range(iters):
    startboard = np.asarray(gen_board(rows, cols, bots))
    endboard = np.asarray(gen_board(rows, cols, bots))
    try:
      blackbox(startboard, endboard)
    except (RuntimeError, IndexError, TypeError, Exception):
      count += 1
  return count

In [ ]:
results = {}
for i in range(2,7):
  start = time.time()
  results[i] = rando_test(5, 5, i, 100000) / 1000
  print(i, time.time() - start)
print(results)

2 34.768983125686646
3 18.617435932159424
4 19.203888177871704


KeyboardInterrupt: 

In [ ]:
plt.plot(results.keys(), results.values())
plt.title("Percentage of random pathfinding trials failed; 100,000 trials on 5x5 grid")
plt.ylabel("Percentage of trials failed, 100,000 trials")
plt.xlabel("Bots")
plt.xticks([2,3,4,5,6])
plt.show()

In [ ]:
for i in range(100000):
  startboard = np.asarray(gen_board(5, 5, 3))
  endboard = np.asarray(gen_board(5, 5, 3))
  try:
    blackbox(startboard, endboard)
  except (RuntimeError, IndexError, TypeError, Exception):
    count += 1
    savestartboard = startboard
    saveendboard = endboard
    print(i)
    raise

In [ ]:
print(savestartboard)
print(saveendboard)
print(blackbox(savestartboard, saveendboard))

In [ ]:
startboard = np.asarray(gen_board(5, 5, 9))
endboard = np.asarray(gen_board(5, 5, 9))
print(startboard)
print(endboard)
finalPathTest = blackbox(startboard, endboard)
print(finalPathTest)
# We're probably having issues with given randomly generated boards
# Split the board generation and blackbox into two cells,
# use that to generate problematic boards and test

# Use random generation for different bot numbers and board sizes

In [ ]:
animation(finalPathTest)

In [ ]:
print(blackbox(startboard, endboard))
# animation(blackbox(startboard, endboard))

<h1>OLD CODE</h1>

In [ ]:
# We can probably phase this out for speed at some point

# Class for bots; we use these to track where to go
class Bot:
  def __init__(self, xpos, ypos, id):
    self.xpos = xpos
    self.ypos = ypos
    self.id = id
    self.hasorders = False
    # self.closestdist = -1

In [ ]:
def find_closest_dists(distlist):
  ind_list = []
  for lst in distlist:
    small1, small2 = two_smallest(lst) # two_smallest gets two smallest vals of lst
    ind1 = lst.index(small1)
    start = 0
    if small1 == small2:
      start = ind1+1
    ind2 = lst.index(small2, start)
    ind_list.append((ind1, ind2))
  return ind_list

In [ ]:
print(find_closest_dists(distlist))
# Need to somehow convert to finding smallest indices of each list
# Can assume 0,1,2,3,4 for now to work on other functionality

In [ ]:
# Rather than make an array of bots, faster to just list of bots and positions
# The list of bots only used to perform operations when bot class exists

# Given an array of 1/0, convert to bot/empty
# Returns converted board and ordered list of bots, left to right, top to bottom
def convert_board(board, xsize, ysize):
  id = 0
  botlist = []
  newboard = [[0]*xsize for i in range(ysize)]
  for row in range(ysize):
    for col in range(xsize):
      if board[row, col] == 1:
        bot = Bot(col, row, id)
        newboard[row][col] = bot
        id += 1
        botlist.append(bot)
      else:
        newboard[row][col] = None
  return newboard, botlist

In [ ]:
# Once bots have closest place to go to, loop through bots,
  # do a move in positive direction

# There is no mechanism for a bot that gets stuck; need to add that later
# Though, that could also be handled in pathing stage
def perform_move(poslist, endlist, distlist):
  # step_count exclusively to track performance
  step_count = 0
  while(poslist is not endlist):
    step_count += 1
    for i in range(len(poslist)):
      # The move:
      # ith starting position
      pos = poslist[i]
      # distlist[i] = index of end pos for bot i
      end = endlist[distlist[i]]
      if pos is not end:
        sx, sy = pos
        ex, ey = end
        if sx < ex and (sx+1, sy) not in poslist:
          poslist[i] = (sx+1, sy)
        elif sx > ex and (sx-1, sy) not in poslist:
          poslist[i] = (sx-1, sy)
        elif sy < ey and (sx, sy+1) not in poslist:
          poslist[i] = (sx, sy+1)
        elif sy > ey and (sx, sy-1) not in poslist:
          poslist[i] = (sx, sy-1)

In [ ]:
# Pretty sure this is exact code for itertools.product

def product(arr):
  pools = [tuple(pool) for pool in arr]
  result = [[]]
  for pool in pools:
    result = [x+[y] for x in result for y in pool]
  for prod in result:
    yield tuple(prod)

In [ ]:
# Given dict of list of lists of possible paths for each bot,
# brute force find first path with no collisions
def find_best_path(paths):
  return
  # (Maybe) better idea: take two bots, remove all collision paths
  # Then, add third bot and calculate with reduced path list
  # for bots in range(2,paths.items().len+1):

  # OR, OVA? But then we can't figure out what path to remove

  # Problem: Collision happens from pair of paths,
  # but we have no way of knowing if either path is good elsewhere
  # Maybe then we need to do more brute force?

  # OVO to condense length could work?
  # So, bot i and i+1 compare, keep only path combos with no collisions
  # Then, we have list of tuples of positions, compare to other list of tuples?

  # Could also do:
  # Organize paths based on current locations; i.e.,
  # make groups for each bot and compare those groups with other bots
  # Instead of checking 12 paths with pos (x1,x2) with 13 paths with pos (y1,y2),
  # we check if (x1,x2)==(y1,y2) and judge the whole group from that

In [ ]:
# currpos is dict from id to current position
# goals is dict from id to end position
def perform_move(currpos, goals):
  # From current position, move each bot one position towards end goal

  # Keep list of new positions for each bot,
  # before actually moving, check for duplicate values,
  # Unlikely to be triples (maybe ignore for now?)
  # determine which direction is best for each bot (how?)

  # Inevitably, we'll have issue where we go the wrong way
  # Ex: bot top left to center, goes right first, should go down first
  return
  # Generate board from newpos after movement

In [ ]:
# Actual movement loop
goals = convert_goals(minsum(arr), endlist)

# When going into application, add waits to allow for acutation time